# Set up

In [ ]:
# Install required packages
!pip3 install openpyxl simpy

In [ ]:
# Import libraries
import pandas as pd
import simpy
import numpy as np
import math
from collections import defaultdict
import datetime
import matplotlib.pyplot as plt
import copy
from datetime import datetime, timedelta
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load Excel file
df_supply = pd.read_excel("/content/drive/MyDrive/IEOR 4418 Project/data/CUL October 2025 Supply Plan.xlsx")

# Column mapping based on Excel structure
# Columns: Date, Day, 4p-5p, 5p-6p, 6p-7p, 7p-8p, 8p-9p, 9p-10p, 10p-11p, 11p-12a, 12a-1a, 1a-2a, 2a-3a, Total, Drivers @ Peak
hour_columns = {
    16: '4p-5p',
    17: '5p-6p',
    18: '6p-7p',
    19: '7p-8p',
    20: '8p-9p',
    21: '9p-10p',
    22: '10p-11p',
    23: '11p-12a',
    0: '12a-1a',
    1: '1a-2a',
    2: '2a-3a',
}

In [ ]:
# Convert to dictionary
supply_plan = {}

for _, row in df_supply.iterrows():
    # Parse date
    date = pd.to_datetime(row['Date']).date()
    supply_plan[date] = {}

    for hour, col_name in hour_columns.items():
        if col_name in df_supply.columns:
            vehicles = row[col_name]
            if pd.isna(vehicles):
                vehicles = 16  # Default
            supply_plan[date][hour] = int(vehicles)

print(f"  Loaded supply plan for {len(supply_plan)} dates")

  Loaded supply plan for 31 dates


In [ ]:
# Load via data
df = pd.read_csv("/content/drive/MyDrive/IEOR 4418 Project/data/Ride Requests_2025-11-11-part-2.csv")
df['Request_Time'] = pd.to_datetime(df['Request Creation Time'])
if 'Request Status' in df.columns:
    df = df[df['Request Status'] == 'Completed']
print(f"  Filtered to {len(df)} completed rides")
print(f"  Date range: {df['Request_Time'].min()} to {df['Request_Time'].max()}")

  Filtered to 27257 completed rides
  Date range: 2025-09-15 19:59:00 to 2025-10-31 23:57:00


In [ ]:
# Create total travel time column, which is actual dropoff time - Actual Pickup Time
df['Travel_Time'] = pd.to_datetime(df['Actual Dropoff Time']) - pd.to_datetime(df['Actual Pickup Time'])
df['Wait Time'] = pd.to_datetime(df['Actual Pickup Time']) - pd.to_datetime(df['Request Creation Time'])
df['Travel_Time'] = df['Travel_Time'].dt.total_seconds() / 60
df['Wait_Time'] = df['Wait Time'].dt.total_seconds() / 60
df.head()

,Request Creation Time,Request Status,Number of Passengers,On Demand Pickup ETA Minutes,Origin Lat,Origin Lng,Destination Lat,Destination Lng,Actual Pickup Time,Actual Dropoff Time,Request_Time,Travel_Time,Wait Time,Wait_Time
2,9/15/2025 19:59,Completed,1,11.9,40.800520,-73.967568,40.813341,-73.961237,9/15/2025 20:18,9/15/2025 20:32,2025-09-15 19:59:00,14.0,0 days 00:19:00,19.0
4,9/15/2025 20:00,Completed,1,10.0,40.818184,-73.959134,40.808708,-73.958553,9/15/2025 20:16,9/15/2025 20:30,2025-09-15 20:00:00,14.0,0 days 00:16:00,16.0
5,9/15/2025 20:00,Completed,1,9.4,40.817832,-73.958271,40.803711,-73.964770,9/15/2025 20:15,9/15/2025 20:26,2025-09-15 20:00:00,11.0,0 days 00:15:00,15.0
7,9/15/2025 20:00,Completed,1,33.7,40.816860,-73.960385,40.809748,-73.959489,9/15/2025 20:36,9/15/2025 20:42,2025-09-15 20:00:00,6.0,0 days 00:36:00,36.0
8,9/15/2025 20:00,Completed,1,16.6,40.805243,-73.954770,40.800588,-73.968130,9/15/2025 20:26,9/15/2025 20:36,2025-09-15 20:00:00,10.0,0 days 00:26:00,26.0


In [ ]:
# Average
avg_travel_time = df['Travel_Time'].mean()
avg_wait_time = df['Wait_Time'].mean()
print(f"  Average travel time: {avg_travel_time:.2f} minutes")
print(f"  Average wait time: {avg_wait_time:.2f} minutes")

  Average travel time: 8.97 minutes
  Average wait time: 23.64 minutes


In [ ]:
print("Filtering to service hours (6 PM - 3 AM)...")

def get_service_date_and_hour(dt):
    """Map a datetime to the service date and hour."""
    if dt.hour >= 18:  # 6 PM onwards - same day
        return dt.date(), dt.hour
    elif dt.hour < 3:  # Midnight to 3 AM - previous day's service
        return (dt - timedelta(days=1)).date(), dt.hour
    else:
        return None, None  # Outside service hours


# Apply the function and create separate columns properly
service_info = df['Request_Time'].apply(get_service_date_and_hour)
df['service_date'] = service_info.apply(lambda x: x[0])
df['service_hour'] = service_info.apply(lambda x: x[1])

df_service = df[df['service_date'].notna()].copy()
print(f"  Requests during service hours (before filtering): {len(df_service)}")
print(f"  Ride data date range: {df_service['service_date'].min()} to {df_service['service_date'].max()}")
print(f"  Supply plan date range: {min(supply_plan.keys())} to {max(supply_plan.keys())}")

# Filter to only dates in supply plan (October 2025)
supply_min_date = min(supply_plan.keys())
supply_max_date = max(supply_plan.keys())
df_service = df_service[(df_service['service_date'] >= supply_min_date) &
                        (df_service['service_date'] <= supply_max_date)].copy()
print(f"\n  Filtered to supply plan dates (October 2025)")
print(f"  Requests after filtering: {len(df_service)}")
print(f"  Filtered date range: {df_service['service_date'].min()} to {df_service['service_date'].max()}")

Filtering to service hours (6 PM - 3 AM)...
  Requests during service hours (before filtering): 27093
  Ride data date range: 2025-09-15 to 2025-10-31
  Supply plan date range: 2025-10-01 to 2025-10-31

  Filtered to supply plan dates (October 2025)
  Requests after filtering: 18286
  Filtered date range: 2025-10-01 to 2025-10-31


In [ ]:
# Define corner grid (streets and avenues)
streets = [103, 106, 110, 113, 116, 120, 122, 125, 129, 133,135] #list(range(103, 136,3))
avenues = {
    "Riverside": -73.9704,
    "Broadway": -73.9626,
    "Amsterdam": -73.9656,
    "Morningside": -73.9586,
    "StNicholas": -73.9498
}

def street_to_lat(st):
    return (st - 100)*0.0009 + 40.800

corners = []
for st in streets:
    lat = street_to_lat(st)
    for name, lon in avenues.items():
        corners.append((lat, lon))

print(f"Total corners defined: {len(corners)}")

Total corners defined: 55


# Config & Helper Function

In [ ]:
# Configuration
MAX_WALK_MIN = 3.0
SPEED_MPS = 3.57 #8mph
DWELL_TIME_SEC = 120
DISPATCH_FRICTION_SEC = 90


def haversine_m(p1, p2):
    lat1, lon1 = p1
    lat2, lon2 = p2
    R = 6371000
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dl = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dl/2)**2
    return R * 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

def get_nearest_corner(point, corners):
    dists = [haversine_m(point, c) for c in corners]
    min_idx = int(np.argmin(dists))
    return corners[min_idx], dists[min_idx]

def calibrated_travel_time(p1, p2, is_corner_trip=False):
    dist = haversine_m(p1, p2)

    if is_corner_trip:
        effective_dist = dist * 1.1
        effective_speed = SPEED_MPS * 1.2
    else:
        effective_dist = dist * 1.3
        effective_speed = SPEED_MPS

    base_time = effective_dist / effective_speed

    # Add Traffic Lights
    num_blocks = dist / 250  # NYC blocks are ~250m
    num_lights = num_blocks  # Not every intersection has lights
    avg_red_wait = 40  # Average wait when hitting a red light
    prob_red = 0.45  # 30% chance of hitting red

    traffic_light_delay = num_lights * avg_red_wait * prob_red

    # 70% of time: Normal traffic (0-10% delay)
    # 20% of time: Moderate traffic (20-50% delay)
    # 10% of time: Bad congestion (50-200% delay)
    rand = random.random()
    if rand < 0.5:
        congestion_factor = random.uniform(1.0, 1.1)
    else:
        congestion_factor = random.uniform(1.3, 2)

    return (base_time * congestion_factor) + traffic_light_delay

def calculate_route_time(current_loc, route):
    if not route: return 0
    total = 0
    last = current_loc
    for stop in route:
        lat, lon, _, _ = stop
        target = (lat, lon)
        start_is_corner = last in corners
        end_is_corner = target in corners
        is_avenue_trip = start_is_corner and end_is_corner
        total += calibrated_travel_time(last, target, is_corner_trip=is_avenue_trip)
        total += DWELL_TIME_SEC
        total += DISPATCH_FRICTION_SEC
        last = target
    return total

print("  Helper functions defined")

  Helper functions defined


# Vehicle Class

In [ ]:
class Vehicle:
    def __init__(self, env, id, start_loc, tracker, max_wait_guarantee=20):
        self.env = env
        self.id = id
        self.tracker = tracker
        self.location = start_loc
        self.capacity = random.randint(3, 6)
        self.passengers = set()
        self.route = []
        self.active_stop = None
        self.max_wait_guarantee = max_wait_guarantee
        self.action = env.process(self.run())

    def run(self):
        while True:
            if not self.route:
                yield self.env.timeout(1)
                continue

            next_stop = self.route[0]
            target_loc = (next_stop[0], next_stop[1])

            # Batch all stops at same location
            batch = []
            while self.route and (self.route[0][0], self.route[0][1]) == target_loc:
                batch.append(self.route.pop(0))

            # Calculate drive time
            start_is_corner = self.location in corners
            end_is_corner = target_loc in corners
            is_avenue_trip = start_is_corner and end_is_corner

            drive_time = calibrated_travel_time(self.location, target_loc, is_corner_trip=is_avenue_trip)

            # Execute drive + friction
            yield self.env.timeout(drive_time + DISPATCH_FRICTION_SEC)
            self.location = target_loc

            # Reduced dwell time for fairness
            total_dwell = DWELL_TIME_SEC + (15 * max(0, len(batch) - 1))
            yield self.env.timeout(total_dwell)

            timestamp = self.env.now

            # Process pickups and dropoffs
            for _, _, action, req_id in batch:
                if action == 'pickup':
                    self.passengers.add(req_id)
                    if req_id in self.tracker:
                        actual_wait = (timestamp - self.tracker[req_id]['req_time']) / 60
                        self.tracker[req_id]['pickup_time'] = timestamp
                        self.tracker[req_id]['actual_wait'] = actual_wait
                elif action == 'dropoff':
                    if req_id in self.passengers:
                        self.passengers.remove(req_id)
                        if req_id in self.tracker:
                            self.tracker[req_id]['dropoff_time'] = timestamp

            self.active_stop = None

print("  Vehicle class defined")

  Vehicle class defined


# Dispatcher

In [ ]:
def check_capacity(initial_load, route, capacity=6):
    """
    Simulates the route to ensure passenger count never exceeds capacity.
    """
    current_load = initial_load
    for _, _, action, _ in route:
        if action == 'pickup':
            current_load += 1
        elif action == 'dropoff':
            current_load -= 1

        # If load ever exceeds capacity, this route is invalid
        if current_load > capacity:
            return False
    return True

def find_best_vehicle_fair(fleet, request, current_time, max_wait_minutes=20, max_ride_minutes=30): # Added max_ride_minutes
    best_veh = None
    best_route = None
    min_cost = float('inf')

    p_loc = request['pickup_loc']
    d_loc = request['dropoff_loc']
    rid = request['id']
    req_time = request.get('req_time', 0)

    direct_distance = haversine_m(p_loc, d_loc)

    # Cost Weights
    ALPHA_QUEUE = 5.0
    BETA_WAIT = 4.0
    GAMMA_DETOUR = 3.0

    for v in fleet:
        if len(v.route) > 5: # Max Queue length constraint
            continue

        # Calculate current load based on passengers currently in the car
        current_load = len(v.passengers)

        curr_route = v.route[:]
        start_node = v.active_stop if v.active_stop else (*v.location, None, None)
        current_pos = (start_node[0], start_node[1])

        # Iterate through insertion points
        for i in range(len(curr_route) + 1):
            j = i + 1
            cand = curr_route[:]
            cand.insert(i, (p_loc[0], p_loc[1], 'pickup', rid))
            cand.insert(j, (d_loc[0], d_loc[1], 'dropoff', rid))

            # Check if this specific route permutation violates capacity
            if not check_capacity(current_load, cand, capacity=v.capacity):
                continue

            valid_assignment = True
            simulated_time = current_time
            last_loc = current_pos
            temp_tracker = {} # To track timestamps for ride time checks

            # Simulate the timeline
            for stop in cand:
                lat, lon, action, stop_req_id = stop
                target = (lat, lon)

                # Travel time logic (same as before)
                dist = haversine_m(last_loc, target)
                drive_sec = (dist / SPEED_MPS) * 1.6 # Safety buffer
                simulated_time += drive_sec + 90 + 60 # Dwell + Friction

                if action == 'pickup':
                    temp_tracker[stop_req_id] = {'pickup': simulated_time}

                    # Wait Time Constraint
                    p_req_time = req_time if stop_req_id == rid else v.tracker[stop_req_id]['req_time']
                    if (simulated_time - p_req_time) / 60 > max_wait_minutes:
                        valid_assignment = False
                        break

                elif action == 'dropoff':
                    # --- FIX 2: RIDE TIME CONSTRAINT ---
                    # We must ensure the ride duration isn't too long
                    pickup_ts = temp_tracker.get(stop_req_id, {}).get('pickup')

                    # If we don't have pickup time (passenger already in car), estimate it
                    if pickup_ts is None and stop_req_id in v.passengers:
                         # Use current time as proxy or check tracker if available
                         pickup_ts = v.tracker[stop_req_id].get('pickup_time', current_time)

                    if pickup_ts:
                        ride_min = (simulated_time - pickup_ts) / 60
                        if ride_min > max_ride_minutes:
                            valid_assignment = False
                            break

                last_loc = target

            if not valid_assignment:
                continue

            # --- Cost Calculation (same as before) ---
            new_pass_pickup = temp_tracker[rid]['pickup']
            new_wait_min = (new_pass_pickup - req_time) / 60

            queue_penalty = len(cand) * ALPHA_QUEUE
            wait_penalty = new_wait_min * BETA_WAIT

            # Detour penalty
            ride_distance = haversine_m(p_loc, d_loc)
            detour_penalty = ((ride_distance / max(direct_distance, 1)) - 1.0) * GAMMA_DETOUR

            total_cost = queue_penalty + wait_penalty + detour_penalty

            if total_cost < min_cost:
                min_cost = total_cost
                best_veh = v
                best_route = cand

    return best_veh, best_route

# Simulation Function(1 hour)

In [ ]:
def run_simulation_for_hour(df_hour, num_vehicles, scenario_mode='hybrid'):
    """
    GUARANTEED SERVICE LEVEL: Every student gets picked up within 40 minutes.

    Approach: Spawn overflow vehicles PROACTIVELY based on predicted wait times.
    """
    env = simpy.Environment()
    tracker = {}
    # Start with base fleet
    fleet = [Vehicle(env, i, corners[i%len(corners)], tracker, max_wait_guarantee=20)
             for i in range(num_vehicles)]

    sorted_reqs = df_hour.sort_values('Request_Time')
    hour_start = sorted_reqs['Request_Time'].min()
    hour_start = hour_start.replace(minute=0, second=0, microsecond=0)

    corner_users = 0
    total_users = 0
    overflow_vehicles = []

    MAX_WAIT_SLA = 50   # If > 40, we count it as a "System Failure"
    MAX_RIDE_SLA = 30   # If > 30, we count it as a "System Failure"

    # Strict service level agreement
    TARGET_WAIT = 30
    TARGET_RIDE = 15

    for idx, row in sorted_reqs.iterrows():
        req_time = (row['Request_Time'] - hour_start).total_seconds()

        if req_time > env.now:
            env.run(until=req_time)

        # Prepare request
        origin = (row['Origin Lat'], row['Origin Lng'])
        dest = (row['Destination Lat'], row['Destination Lng'])

        p_corn, p_dist = get_nearest_corner(origin, corners)
        d_corn, d_dist = get_nearest_corner(dest, corners)
        p_walk_min = (p_dist / 1.4) / 60
        d_walk_min = (d_dist / 1.4) / 60

        use_corner = False
        if scenario_mode == 'corner':
            use_corner = True
        elif scenario_mode == 'door':
            use_corner = False
        elif scenario_mode == 'hybrid':
            if (p_walk_min <= MAX_WALK_MIN) and (d_walk_min <= MAX_WALK_MIN):
                use_corner = True

        if use_corner:
            pickup, dropoff = p_corn, d_corn
            walk_time = (p_walk_min + d_walk_min) * 60
            corner_users += 1
        else:
            pickup, dropoff = origin, dest
            walk_time = 0

        total_users += 1

        tracker[idx] = {
            'req_id': idx,
            'req_time': env.now,
            'walk_time': walk_time,
            'service_type': 'corner' if use_corner else 'door',
            'pickup_time': None,
            'dropoff_time': None,
            'actual_wait': None,
            'status': 'pending'
        }

        req = {
            'id': idx,
            'pickup_loc': pickup,
            'dropoff_loc': dropoff,
            'req_time': env.now
        }

        # Combine base fleet + overflow vehicles
        all_vehicles = fleet + overflow_vehicles

        # Try assignment with STRICT SLA
        veh, route = find_best_vehicle_fair(all_vehicles, req, env.now,
                                            max_wait_minutes=TARGET_WAIT,  # Dispatcher rejects if > 30
                                            max_ride_minutes=TARGET_RIDE)

        if veh and route:
            veh.route = route
            tracker[idx]['status'] = 'served'
        else:
            # SLA guarantees that Spawn dedicated vehicle immediately
            new_overflow = Vehicle(env, len(fleet) + len(overflow_vehicles),
                                  pickup, tracker, max_wait_guarantee=MAX_WAIT_SLA)
            new_overflow.route = [
                (pickup[0], pickup[1], 'pickup', idx),
                (dropoff[0], dropoff[1], 'dropoff', idx)
            ]
            overflow_vehicles.append(new_overflow)
            tracker[idx]['status'] = 'served'

    # Run simulation
    env.run(until=env.now + 3600)

    # Compile results
    results = []

    for rid, data in tracker.items():
        if data['status'] == 'served' and data['pickup_time'] and data['dropoff_time']:
            wait = (data['pickup_time'] - data['req_time']) / 60
            ride = (data['dropoff_time'] - data['pickup_time']) / 60
            walk = data['walk_time'] / 60

            # Check for violations,
            is_wait_violation = wait > MAX_WAIT_SLA
            is_ride_violation = ride > MAX_RIDE_SLA

            results.append({
                'scenario': scenario_mode,
                'service_type': data['service_type'],
                'wait_min': wait,   # Record the REAL wait time
                'ride_min': ride,   # Record the REAL ride time
                'walk_min': walk,
                'total_min': wait + ride,
                'violation': is_wait_violation or is_ride_violation # Boolean flag
            })

    return pd.DataFrame(results), corner_users, total_users

print("  Simulation function defined")

  Simulation function defined


# Main Loop

In [ ]:
# Group by service date and hour
grouped = df_service.groupby(['service_date', 'service_hour'])

all_results = []
summary_results = []

total_combinations = len(grouped)
processed = 0

for (service_date, service_hour), group_df in grouped:
    processed += 1

    # Convert to int to avoid formatting issues
    service_hour = int(service_hour)

    # Get vehicle count from supply plan
    if service_date in supply_plan and service_hour in supply_plan[service_date]:
        num_vehicles = supply_plan[service_date][service_hour]
    else:
        num_vehicles = 16  # Default

    # Use FULL fleet (no reduction) to prevent overflow
    num_vehicles = max(1, int(num_vehicles * 1.0))  # Use 100% of planned fleet
    n_requests = len(group_df)

    # Skip if too few requests
    if n_requests < 5:
        print(f"\n[{processed}/{total_combinations}] {service_date} {service_hour:02d}:00 - SKIPPED (only {n_requests} requests)")
        continue

    print(f"\n[{processed}/{total_combinations}] {service_date} {service_hour:02d}:00-{(service_hour+1)%24:02d}:00")
    print(f"  Requests: {n_requests}, Vehicles: {num_vehicles}")

    # Run three scenarios
    try:
        res_door, corner_door, total_door = run_simulation_for_hour(group_df, num_vehicles, 'door')
        res_corner, corner_corner, total_corner = run_simulation_for_hour(group_df, num_vehicles, 'corner')
        res_hybrid, corner_hybrid, total_hybrid = run_simulation_for_hour(group_df, num_vehicles, 'hybrid')

        # Add metadata
        for df_res, scenario in [(res_door, 'door'), (res_corner, 'corner'), (res_hybrid, 'hybrid')]:
            df_res['service_date'] = service_date
            df_res['service_hour'] = service_hour
            df_res['num_vehicles'] = num_vehicles
            df_res['total_requests'] = n_requests
            all_results.append(df_res)

        # Summary statistics
        summary_results.append({
            'date': service_date,
            'hour': service_hour,
            'n_vehicles': num_vehicles,
            'n_requests': n_requests,
            'door_avg_wait': res_door['wait_min'].mean() if len(res_door) > 0 else None,
            'door_avg_total': res_door['total_min'].mean() if len(res_door) > 0 else None,
            'corner_avg_wait': res_corner['wait_min'].mean() if len(res_corner) > 0 else None,
            'corner_avg_total': res_corner['total_min'].mean() if len(res_corner) > 0 else None,
            'hybrid_avg_wait': res_hybrid['wait_min'].mean() if len(res_hybrid) > 0 else None,
            'hybrid_avg_total': res_hybrid['total_min'].mean() if len(res_hybrid) > 0 else None,
            'hybrid_corner_pct': (corner_hybrid / total_hybrid * 100) if total_hybrid > 0 else 0
        })

        print(f"  ✅ Door: {res_door['wait_min'].mean():.1f}min wait, {res_door['total_min'].mean():.1f}min total")
        print(f"  ✅ Corner: {res_corner['wait_min'].mean():.1f}min wait, {res_corner['total_min'].mean():.1f}min total")
        print(f"  ✅ Hybrid: {res_hybrid['wait_min'].mean():.1f}min wait, {res_hybrid['total_min'].mean():.1f}min total ({corner_hybrid/total_hybrid*100:.1f}% corner)")

    except Exception as e:
        print(f"  ❌ Error: {e}")
        continue



[1/276] 2025-10-01 00:00-01:00
  Requests: 55, Vehicles: 10
  ✅ Door: 19.5min wait, 37.9min total
  ✅ Corner: 18.6min wait, 31.6min total
  ✅ Hybrid: 17.6min wait, 34.5min total (65.5% corner)

[2/276] 2025-10-01 01:00-02:00
  Requests: 23, Vehicles: 6
  ✅ Door: 19.7min wait, 40.1min total
  ✅ Corner: 15.1min wait, 27.4min total
  ✅ Hybrid: 14.9min wait, 30.5min total (69.6% corner)

[3/276] 2025-10-01 02:00-03:00
  Requests: 23, Vehicles: 4
  ✅ Door: 19.0min wait, 35.1min total
  ✅ Corner: 14.3min wait, 28.6min total
  ✅ Hybrid: 17.7min wait, 34.0min total (56.5% corner)

[4/276] 2025-10-01 18:00-19:00
  Requests: 72, Vehicles: 7
  ✅ Door: 20.5min wait, 42.3min total
  ✅ Corner: 19.6min wait, 35.0min total
  ✅ Hybrid: 19.5min wait, 39.0min total (40.3% corner)

[5/276] 2025-10-01 19:00-20:00
  Requests: 84, Vehicles: 8
  ✅ Door: 21.5min wait, 40.9min total
  ✅ Corner: 20.1min wait, 35.6min total
  ✅ Hybrid: 20.1min wait, 38.1min total (50.0% corner)

[6/276] 2025-10-01 20:00-21:00
  

# Result and Analysis

In [ ]:
df_all_path = "/content/drive/MyDrive/IEOR 4418 Project/output_data/full_hourly_results_detailed.csv"
df_summary_path = "/content/drive/MyDrive/IEOR 4418 Project/output_data/full_hourly_results_summary.csv"

if all_results:
    df_all = pd.concat(all_results, ignore_index=True)


    df_summary = pd.DataFrame(summary_results)

    # Save to CSV
    df_all.to_csv(df_all_path, index=False)
    df_summary.to_csv(df_summary_path, index=False)

    print(f"\n Saved {len(df_all)} detailed results to full_hourly_results_detailed.csv")
    print(f"Saved {len(df_summary)} hourly summaries to full_hourly_results_summary.csv")

    # Print overall statistics
    print("\n" + "="*80)
    print("OVERALL STATISTICS (OUTLIERS REMOVED)")
    print("="*80)

    for scenario in ['door', 'corner', 'hybrid']:
        df_scenario = df_all[df_all['scenario'] == scenario]
        if len(df_scenario) > 0:
            print(f"\n{scenario.upper()}:")
            print(f"  Total rides: {len(df_scenario)}")
            print(f"  Avg wait: {df_scenario['wait_min'].mean():.2f} min")
            print(f"  Avg ride: {df_scenario['ride_min'].mean():.2f} min")
            print(f"  Avg walk: {df_scenario['walk_min'].mean():.2f} min")
            print(f"  Avg total: {df_scenario['total_min'].mean():.2f} min")

    # Hybrid breakdown
    df_hybrid = df_all[df_all['scenario'] == 'hybrid']
    if len(df_hybrid) > 0:
        print(f"\nHYBRID BREAKDOWN:")
        for service_type in ['door', 'corner']:
            df_type = df_hybrid[df_hybrid['service_type'] == service_type]
            if len(df_type) > 0:
                print(f"  {service_type.upper()} users: {len(df_type)} ({len(df_type)/len(df_hybrid)*100:.1f}%)")
                print(f"    Avg wait: {df_type['wait_min'].mean():.2f} min")
                print(f"    Avg total: {df_type['total_min'].mean():.2f} min")

else:
    print("\n No results generated")



 Saved 54072 detailed results to full_hourly_results_detailed.csv
Saved 276 hourly summaries to full_hourly_results_summary.csv

OVERALL STATISTICS (OUTLIERS REMOVED)

DOOR:
  Total rides: 17796
  Avg wait: 19.17 min
  Avg ride: 19.33 min
  Avg walk: 0.00 min
  Avg total: 38.50 min

CORNER:
  Total rides: 18253
  Avg wait: 15.93 min
  Avg ride: 15.14 min
  Avg walk: 4.57 min
  Avg total: 31.08 min

HYBRID:
  Total rides: 18023
  Avg wait: 17.79 min
  Avg ride: 17.76 min
  Avg walk: 1.84 min
  Avg total: 35.55 min

HYBRID BREAKDOWN:
  DOOR users: 7674 (42.6%)
    Avg wait: 17.86 min
    Avg total: 39.02 min
  CORNER users: 10349 (57.4%)
    Avg wait: 17.74 min
    Avg total: 32.98 min
